In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [ ]:
def generar_features_temporales(df, ventana_rolling=3):
    """
    Genera características relacionadas con series temporales a partir de un DataFrame con 
    DateTimeIndex.
    
    Parámetros:
    df (pd.DataFrame): DataFrame de entrada con índice de tipo DateTimeIndex.
    ventana_rolling (int): Tamaño de la ventana para las estadísticas móviles (por defecto 3).
    
    Retorno:
    pd.DataFrame: DataFrame con las características temporales agregadas.
    """
    
    # Verificar si el índice es de tipo DateTime
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("El DataFrame debe tener un índice de tipo DateTimeIndex.")
    
    # Crear nuevas columnas basadas en el índice de fecha
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['day_of_week'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['is_month_start'] = df.index.is_month_start
    df['is_month_end'] = df.index.is_month_end
    df['is_quarter_start'] = df.index.is_quarter_start
    df['is_quarter_end'] = df.index.is_quarter_end
    
    # Crear características de lag (retardo) y estadísticas móviles (rolling)
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        df[f'{col}_lag1'] = df[col].shift(1)
        df[f'{col}_rolling_mean'] = df[col].rolling(window=ventana_rolling).mean()
        df[f'{col}_rolling_std'] = df[col].rolling(window=ventana_rolling).std()

    return df

In [ ]:
df = pd.read_csv("../data/processed/procedimientos_procesada.csv")

df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")
df = df.dropna(subset="fecha")

In [ ]:
produccion_radiografia = (
    df.query("accion_clinica == 'RADIOGRAFIA DE TORAX FRONTAL Y LATERAL'")
    .groupby("fecha")["numero_de_veces"]
    .sum()
)

In [ ]:
produccion_radiografia_mensual = produccion_radiografia.resample("M").sum().to_frame()

In [ ]:
fecha_corte = "2020-01-01"
train = produccion_radiografia_mensual.query("fecha < @fecha_corte").copy()
test = produccion_radiografia_mensual.query("fecha >= @fecha_corte").copy()

In [ ]:
train = generar_features_temporales(train)
test = generar_features_temporales(test)

train = train.dropna()
test = test.dropna()

# Separa en X_train y y_train
X_train = train.drop(columns="numero_de_veces")
y_train = train["numero_de_veces"]

X_test = test.drop(columns="numero_de_veces")
y_test = test["numero_de_veces"]

In [ ]:
modelo = LinearRegression(fit_intercept=True)

In [ ]:
modelo.fit(X_train, y_train)

In [ ]:
yhat = modelo.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
round(mean_absolute_error(y_test, yhat))